In [27]:
import pandas as pd
import numpy as np
import os
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
import faiss
from typing import List
from langchain.embeddings.base import Embeddings
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [2]:
data = pd.read_csv('KnowledgeBase.csv')
data.head()

,Топик,Подтопик,Поисковой запрос,Текст
0,Главное,Приложение Мой МТС,Как проверить последние пополнения и списания,1. Откройте Расходы\n\n\n2. По умолчанию отоб...
1,Главное,Приложение Мой МТС,Как заказать детализацию,"\nВ детализации есть полные данные о том, как ..."
2,Главное,Приложение Мой МТС,Как оставаться на связи при минусе,Есть несколько способов пользоваться номером п...
3,Главное,Приложение Мой МТС,Как проверить баланс и пакеты по тарифу,"На главном экране, который открывается при вхо..."
4,Главное,Приложение Мой МТС,"Что делать, если остатки пакетов не показываются","Вы не увидите остатки интернета, минут или СМС..."


In [3]:
data.columns = ['topic', 'subtopic', 'query', 'text']
data.head()

,topic,subtopic,query,text
0,Главное,Приложение Мой МТС,Как проверить последние пополнения и списания,1. Откройте Расходы\n\n\n2. По умолчанию отоб...
1,Главное,Приложение Мой МТС,Как заказать детализацию,"\nВ детализации есть полные данные о том, как ..."
2,Главное,Приложение Мой МТС,Как оставаться на связи при минусе,Есть несколько способов пользоваться номером п...
3,Главное,Приложение Мой МТС,Как проверить баланс и пакеты по тарифу,"На главном экране, который открывается при вхо..."
4,Главное,Приложение Мой МТС,"Что делать, если остатки пакетов не показываются","Вы не увидите остатки интернета, минут или СМС..."


In [5]:
data = data.drop('text', axis=1)
data.head()

,topic,subtopic,query
0,Главное,Приложение Мой МТС,Как проверить последние пополнения и списания
1,Главное,Приложение Мой МТС,Как заказать детализацию
2,Главное,Приложение Мой МТС,Как оставаться на связи при минусе
3,Главное,Приложение Мой МТС,Как проверить баланс и пакеты по тарифу
4,Главное,Приложение Мой МТС,"Что делать, если остатки пакетов не показываются"


In [51]:
data['class'] = data['topic'] + '-' + data['subtopic']
data.head()

,topic,subtopic,query,class
0,Главное,Приложение Мой МТС,Как проверить последние пополнения и списания,Главное-Приложение Мой МТС
1,Главное,Приложение Мой МТС,Как заказать детализацию,Главное-Приложение Мой МТС
2,Главное,Приложение Мой МТС,Как оставаться на связи при минусе,Главное-Приложение Мой МТС
3,Главное,Приложение Мой МТС,Как проверить баланс и пакеты по тарифу,Главное-Приложение Мой МТС
4,Главное,Приложение Мой МТС,"Что делать, если остатки пакетов не показываются",Главное-Приложение Мой МТС


In [10]:
data['class'].unique()

array(['Главное Приложение Мой МТС', 'Главное Личный кабинет',
       'Главное МТС ID', 'Главное МТС Premium', 'Главное МТС Cashback',
       'Безопасность Защитник', 'Безопасность Membrana',
       'Безопасность Мой поиск', 'Развлечения KION',
       'Развлечения МТС Музыка', 'Развлечения МТС Live',
       'Развлечения Строки', "Развлечения МТС GOOD'OK",
       'Развлечения МТС Юрент', 'Развлечения Rushbe',
       'Утилиты Вторая Память', 'Утилиты МТС Лончер', 'Финансы МТС Банк',
       'Финансы МТС Деньги', 'Финансы МТС Все страховки',
       'Финансы МТС Инвестиции', 'Детям и родителям МТС Лончер Junior',
       'Детям и родителям МТС Junior', 'Детям и родителям Membrana Kids',
       'Для дома МТС Умный дом', 'Для дома VDome',
       'Другое Интернет-магазин', 'Другое МТС Тестирование',
       'Другое Crowdloud'], dtype=object)

In [36]:
class MTS_Embeddings(Embeddings):
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_url = "https://api.gpt.mws.ru/v1/embeddings"
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._get_embedding(text) for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        return self._get_embedding(text)
        
    @retry(wait=wait_random_exponential(min=20, max=50), stop=stop_after_attempt(200))
    def _get_embedding(self, text: str) -> List[float]:
        headers = {'Authorization': f'Bearer {self.api_key}'}
        data = {
            "model": "bge-m3",
            'input': text
            }
        try:
            response = requests.post(self.api_url, json=data, headers=headers)
            return response.json()['data'][0]['embedding']
        except Exception as e:
            print(e)
            print(response)
            return e

In [37]:
# Class to normalize embeddings for cosine similarity
class NormalizedEmbeddings(Embeddings):
    def __init__(self, embeddings: Embeddings):
        self.embeddings = embeddings

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.embeddings.embed_documents(texts)
        return [list(np.array(emb) / np.linalg.norm(emb)) for emb in embeddings]

    def embed_query(self, text: str) -> List[float]:
        embedding = self.embeddings.embed_query(text)
        return list(np.array(embedding) / np.linalg.norm(embedding))

In [38]:
text_series = data['query']

In [39]:
api_key = os.getenv('API_KEY')
mts_embeddings = MTS_Embeddings(api_key=api_key)

In [40]:
normalized_embeddings = NormalizedEmbeddings(mts_embeddings)

In [41]:
docs = [Document(page_content=text, metadata={"series_index": idx}) 
        for idx, text in zip(text_series.index, text_series.tolist())]

In [42]:
embeddings = normalized_embeddings.embed_documents([doc.page_content for doc in docs])

In [44]:
dimension = len(embeddings[0])

In [45]:
index = faiss.IndexFlatIP(dimension)

In [46]:
index.add(np.array(embeddings))

In [47]:
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})
index_to_docstore_id = {i: str(i) for i in range(len(docs))}

In [48]:
vector_store = FAISS(
    embedding_function=normalized_embeddings.embed_query,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [ ]:
def retrieve_similar_entry(query: str):
    results = vector_store.similarity_search_with_score(query, k=1)
    if results:
        most_similar_doc, score = results[0]
        original_text = most_similar_doc.page_content
        confidence = score  # Cosine similarity (inner product of normalized vectors)
        original_index = most_similar_doc.metadata["series_index"]
        return {
            "original_text": original_text,
            "confidence": confidence,
        }
    return None

In [52]:
answer_dict = {key : answer for key, answer in zip(data['query'], data['class'])}
answer_dict

{'Как проверить последние пополнения и списания': 'Главное-Приложение Мой МТС',
 'Как заказать детализацию': 'Главное-Приложение Мой МТС',
 'Как оставаться на связи при минусе': 'Главное-Приложение Мой МТС',
 'Как проверить баланс и пакеты по тарифу': 'Главное-Приложение Мой МТС',
 'Что делать, если остатки пакетов не показываются': 'Главное-Приложение Мой МТС',
 'Что такое прогноз списаний и как его посмотреть': 'Главное-Приложение Мой МТС',
 'Почему прогноз может измениться': 'Главное-Приложение Мой МТС',
 'Почему даты списаний могут меняться': 'Главное-Приложение Мой МТС',
 'Как узнать, сколько осталось интернета': 'Главное-Приложение Мой МТС',
 'Что делать, если интернет заканчивается': 'Главное-Приложение Мой МТС',
 'Дополнительные пакеты интернета — автопродление': 'Главное-Приложение Мой МТС',
 'Как узнать подробности о своём тарифе': 'Главное-Приложение Мой МТС',
 'Как настроить тариф': 'Главное-Приложение Мой МТС',
 'Как обновить пакеты по тарифу': 'Главное-Приложение Мой МТС'

In [ ]:
query = 'Чтобы проверить баланс и пакеты по тарифу:\n\n1. **В приложении "Мой МТС":**\n   - На главной странице нажмите на название вашего тарифа.\n   - Откроется экран с информацией о размере и остатках пакетов интернета, минут и SMS.\n   - Нажмите на "Настроить" (для тарифов с пакетами) или "О тарифе" для полного описания.\n\n2. **В Личном кабинете:**\n   - Проверьте подаренные минуты и уточните абонентскую плату.\n\nТакже можно использовать виджеты приложения для уточнения абонентской платы и баланса.'
result = retrieve_similar_entry(query)
if result:
    print(f"Most similar entry: {answer_dict[result['original_text']]}")
    print(f"Confidence (cosine similarity): {result['confidence']}")
else:
    print("No similar entries found.")

Most similar entry: Главное-Приложение Мой МТС
Confidence (cosine similarity): 0.8087646961212158
Original series index: 3
